In [2]:
"""Plot of RHS in L1 error"""

'Plot of RHS in L1 error'

We plot the RHS in the $L_1$ error between $\mathcal{L}_{t}$ and $\mathcal{L}_{t}^{PCA}$

For any $p \geq 1$, plot of

$$ \frac{1}{n} \sum_{i=1}^{n} \Lambda^{i} \, \left(\frac{\sum_{k=3}^{n} \nu^{k} \times (u^{i, k})^{2}}{\sum_{k=1}^{n} \nu^{k} \times (u^{i, k})^{2}}\right)^{\frac{p}{2(p+1)}}. $$

Define also $C_{p} = \frac{9}{\sqrt{\pi}} \Gamma(\frac{1+p}{2})^{\frac{1}{p+1}}$. To simplify, we take $\Lambda^i = 1$ for all $i$.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

In [ ]:
def rhs_func(n, p, b_min, b_max, t=1.0):
    """RHS function for the L1 error bound"""
    b = np.random.uniform(b_min, b_max, size=n)
    b_rep = np.tile(b, (n, 1))
    rho = np.random.uniform(-1, 1, size=n)
    rho_rep = np.tile(rho, (n, 1))
    cov_x = (rho_rep * rho_rep.T) * (1 - np.exp(-(b_rep + b_rep.T) * t))
    cov_x /= b_rep + b_rep.T
    u, eig_x, _ = np.linalg.svd(cov_x)
    num = np.sum(eig_x[2:, None] * u[2:, :]**2, axis=0)
    denom = np.sum(eig_x[:, None] * u[:, :]**2, axis=0)
    res = np.sum(pow(num / denom, 0.5 * p / (p + 1)))
    return res / n


def mc_rhs_func(tab_n, n_mc, p, b_min, b_max):
    """Monte Carlo simulation for the RHS function"""
    bound = np.zeros(tab_n.shape[0])
    bound_err = np.zeros(tab_n.shape[0])

    for idx in tqdm(range(tab_n.shape[0])):
        lst = []
        for _ in range(n_mc):
            _rhs = rhs_func(n=tab_n[idx], p=p, b_min=b_min, b_max=b_max)
            lst.append(_rhs)
        bound[idx] = np.array(lst).mean()
        bound_err[idx] = 1.96 * np.array(lst).std() / np.sqrt(n_mc)
    return bound, bound_err

In [ ]:
N_MC = 30
range_n = np.arange(100, 1500, 100)

In [ ]:
bound_01, bound_err_01 = mc_rhs_func(
    tab_n=range_n,
    n_mc=N_MC,
    p=1,
    b_min=0,
    b_max=1,
)
bound_14, bound_err_14 = mc_rhs_func(
    tab_n=range_n,
    n_mc=N_MC,
    p=1,
    b_min=1,
    b_max=4,
)
bound_010, bound_err_010 = mc_rhs_func(
    tab_n=range_n,
    n_mc=N_MC,
    p=1,
    b_min=0,
    b_max=10,
)

In [ ]:
MARKERSIZE = 10
ELINEWIDTH = 1
CAPSIZE = 3

In [ ]:
fig, ax = plt.subplots()
ax.errorbar(
    range_n,
    bound_01,
    yerr=bound_err_01,
    markersize=MARKERSIZE,
    elinewidth=ELINEWIDTH,
    fmt="*",
    capsize=CAPSIZE,
    label="$b=$ Uniform [0, 1]",
)
ax.errorbar(
    range_n,
    bound_14,
    yerr=bound_err_14,
    markersize=MARKERSIZE,
    elinewidth=ELINEWIDTH,
    fmt=".",
    capsize=CAPSIZE,
    label="$b=$ Uniform [1, 4]",
)
ax.errorbar(
    range_n,
    bound_010,
    yerr=bound_err_010,
    markersize=MARKERSIZE,
    elinewidth=ELINEWIDTH,
    fmt="x",
    capsize=CAPSIZE,
    label="$b=$ Uniform [0, 10]",
)
ax.set_xlabel("$n$")
ax.legend()
plt.show()